In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import albumentations as A
import torch
from torch.utils.data import Dataset, DataLoader

In [5]:
data=pd.read_csv('data/complete_train.csv')
print(data.head())
data_desc=pd.read_csv('data/train_series_descriptions.csv')

   study_id  series_id  instance_number              condition  level  \
0   4003253  702807833                8  Spinal Canal Stenosis  L1/L2   
1   4003253  702807833                8  Spinal Canal Stenosis  L2/L3   
2   4003253  702807833                8  Spinal Canal Stenosis  L3/L4   
3   4003253  702807833                8  Spinal Canal Stenosis  L4/L5   
4   4003253  702807833                8  Spinal Canal Stenosis  L5/S1   

            x           y          image_file_path       output  
0  322.831858  227.964602  4003253_702807833_8.dcm  Normal/Mild  
1  320.571429  295.714286  4003253_702807833_8.dcm  Normal/Mild  
2  323.030303  371.818182  4003253_702807833_8.dcm  Normal/Mild  
3  335.292035  427.327434  4003253_702807833_8.dcm  Normal/Mild  
4  353.415929  483.964602  4003253_702807833_8.dcm  Normal/Mild  


In [8]:
merge_df=pd.merge(data,data_desc,on='series_id',how='inner')
filter_df=merge_df[merge_df['series_description']=='Sagittal T2/STIR']
path_df=filter_df[['image_file_path']].copy()
print(path_df)

                   image_file_path
0          4003253_702807833_8.dcm
1          4003253_702807833_8.dcm
2          4003253_702807833_8.dcm
3          4003253_702807833_8.dcm
4          4003253_702807833_8.dcm
...                            ...
48667  4290709089_3274612423_9.dcm
48668  4290709089_3274612423_9.dcm
48669  4290709089_3274612423_9.dcm
48670  4290709089_3274612423_9.dcm
48671  4290709089_3274612423_9.dcm

[9748 rows x 1 columns]


# datasat

In [ ]:
class SpineDataset(Dataset):
    def __init__(self, df, transforms=None):
        self.df = df
        self.transforms = transforms
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img = load_img(self.df.iloc[index]['image_path'])
        
        # 动态生成mask
        row = self.df.iloc[index]
        mask = generate_mask(row)  # 根据图像信息实时生成mask
        
        if self.transforms:
            augmented = self.transforms(image=img, mask=mask)
            img, mask = augmented['image'], augmented['mask']
        
        return img, mask
